<a href="https://colab.research.google.com/github/MK316/TExams/blob/main/TExamsApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teacher Exam: searching with keywords and displaying question items

In [ ]:
import pandas as pd
from IPython.display import Image, display

url = "https://raw.githubusercontent.com/MK316/TExams/main/TExams.csv"
df = pd.read_csv(url, encoding='utf-8')
df.head()

## Sample

In [ ]:
from IPython.display import display, Image

# Adjusted function to display an image by DataFrame index for URLs
def display_image_by_url(df, index):
    if index < len(df):
        image_url = df.loc[index, 'LINK']  # Replace 'YourColumnNameHere' with the actual column name
        display(Image(url=image_url))
    else:
        print("Index out of range.")

# Example usage
# Ensure df is your DataFrame and it has a column with URLs pointing to the images
# Replace 'YourColumnNameHere' with the actual column name containing the image URLs
display_image_by_url(df, 0)


# Gradio [1]

In [ ]:
%%capture
!pip install gradio

In [ ]:
import gradio as gr

# Sample DataFrame for demonstration purposes
# Replace this with your actual DataFrame
# data = {
#     'Year': ['2024_01'],
#     'ImageURL': ['https://github.com/MK316/TExams/raw/main/images/2024_01.png']  # Replace with your actual URL
# }
# df = pd.DataFrame(data)

url = "https://raw.githubusercontent.com/MK316/TExams/main/TExams.csv"
df = pd.read_csv(url, encoding='utf-8')

# Function to return an HTML image tag with the image URL
def get_image_html(year):
    # Filter the DataFrame for the given year
    match = df[df['YEAR'] == year]
    if not match.empty:
        # Get the first matching URL
        image_url = match.iloc[0]['LINK']
        # Return an HTML image tag with the URL
        return f"<img src='{image_url}' width='800'/>"  # You can adjust the width as needed
    else:
        # Return a message if no match is found
        return "No image found for this year."

# Set up the Gradio interface
iface = gr.Interface(fn=get_image_html,
                     inputs=gr.Textbox(placeholder="Enter Year like '2024_01'"),
                     outputs='html',
                     title="Image Display for a Given Year",
                     description="Type a year and submit to display the corresponding image.")


# Launch the app
iface.launch()


# Gradio [2] (final version)

In [ ]:
import gradio as gr

url = "https://raw.githubusercontent.com/MK316/TExams/main/TExams.csv"
df = pd.read_csv(url, encoding='utf-8')

# Function to return an HTML image tag with the image URL
def get_image_html(year):
    # Filter the DataFrame for the given year
    match = df[df['YEAR'] == year]
    if not match.empty:
        # Get the first matching URL
        image_url = match.iloc[0]['LINK']
        # Return an HTML image tag with the URL
        return f"<img src='{image_url}' width='800'/>"  # You can adjust the width as needed
    else:
        # Return a message if no match is found
        return "No image found for this year."

# Use Gradio's Blocks to create a custom layout with a submit button
def show_image(year, output_component):
    output_component.update(get_image_html(year))

with gr.Blocks() as app:
    gr.Markdown("# 🐳 Teacher Certificate Exam: Display Phonology Questions for a Given Year")
    gr.Markdown("Type a year and submit to display the corresponding image.")
    with gr.Row():
        textbox = gr.Textbox(placeholder="Enter Year like '2024_01' or '2024_02a' if the question is split")
        submit_button = gr.Button("Show the question")
    output = gr.HTML()  # Define the output component outside of the Row
    submit_button.click(fn=get_image_html, inputs=textbox, outputs=output)


# Launch the app
app.launch(debug=True)


# Gradio [3] Searching with keywords added

In [ ]:
import pandas as pd

# Example DataFrame for demonstration purposes

url = "https://raw.githubusercontent.com/MK316/TExams/main/TExams.csv"
df = pd.read_csv(url, encoding='utf-8')

# Function to search for YEARS by matching keywords
def search_year_by_keyword(df, keyword):
    # Create a mask for whether the keyword appears in the 'KEYWORDS' column
    mask = df['KEYWORDS'].apply(lambda x: keyword.lower() in x.lower().split(','))
    # Filter the DataFrame based on the mask and return the 'YEAR' column
    return df[mask]['YEAR'].tolist()

# Example usage:
keyword_to_search = input('Type a keyword: ')  # Replace this with the user's input keyword
matching_years = search_year_by_keyword(df, keyword_to_search)

print(f"Years matching the keyword '{keyword_to_search}': {matching_years}")


# Gradio

In [ ]:
import gradio as gr
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/TExams/main/TExams.csv"
df = pd.read_csv(url, encoding='utf-8')

# Function to search for years by matching keywords

def search_years_by_keywords(keywords):
    # Split the input keywords by commas and strip whitespace
    keyword_list = [keyword.strip() for keyword in keywords.split(',')]
    # Filter the DataFrame rows where any of the keywords matches
    matches = df[df['KEYWORDS'].apply(lambda x: any(keyword in x.split(', ') for keyword in keyword_list))]
    # Return the matched years as a list
    return matches['YEAR'].tolist()

# Create Gradio interface
iface = gr.Interface(
    fn=search_years_by_keywords,
    inputs=gr.Textbox(placeholder="Enter keywords separated by commas"),
    outputs=gr.Text(label="Matching Years"),
    title="Search Years by Keywords",
    description="Type keywords to search for years. Separate keywords with commas."
)

# Launch the app with share=True to get a shareable link
iface.launch(share=True)


# Combining the two applications

In [ ]:
import gradio as gr
import pandas as pd

# Load the DataFrame
url = "https://raw.githubusercontent.com/MK316/TExams/main/TExams.csv"
df = pd.read_csv(url, encoding='utf-8')

# Function to search for years by matching keywords
def search_years_by_keywords(keywords):
    keyword_list = [keyword.strip() for keyword in keywords.split(',')]
    matches = df[df['KEYWORDS'].apply(lambda x: any(keyword in x.split(', ') for keyword in keyword_list))]
    return ", ".join(matches['YEAR'].tolist())  # Return a string of matched years

# Function to return an HTML image tag with the image URL
def get_image_html(year):
    match = df[df['YEAR'] == year]
    if not match.empty:
        image_url = match.iloc[0]['LINK']
        return f"<img src='{image_url}' width='800'/>"
    else:
        return "No image found for this year."

# Create Gradio Blocks interface
with gr.Blocks() as app:
    gr.Markdown("# Teacher Certificate Exam Helper")
    gr.Markdown("## ❄️ [1] Search Years by Keywords")
    keyword_input = gr.Textbox(label = "Keyword", placeholder="Enter keywords separated by commas")
    keyword_output = gr.Text(label="Matching Years")
    gr.Markdown("## ❄️ [2] Display Phonology Questions for a Given Year")
    year_input = gr.Textbox(label="Year information: e.g., Year_item_part(a,b)", placeholder="Enter Year like '2024_01'")
    image_output = gr.HTML()

    # Define actions
    keyword_input.change(fn=search_years_by_keywords, inputs=keyword_input, outputs=keyword_output)
    year_input.change(fn=get_image_html, inputs=year_input, outputs=image_output)

# Launch the app with sharing options
app.launch(share=True)
